In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# GAN MODEL ONLY, Run Cells below to train GAN generate transit Light Curves

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

np.random.seed(1)

print('Loading train and test data...')
df=pd.read_csv('../input/kepler-labelled-time-series-data/exoTrain.csv')
#print(df1)
df2=pd.read_csv('../input/kepler-labelled-time-series-data/exoTest.csv')
#print(df2)

In [ ]:
#print(train_data)


Y=df.loc[df["LABEL"]==2]
Y=df.loc[:,df.columns=="LABEL"]
X=df.loc[df["LABEL"]==2]        #train the GAN on existing exoplanet lightcurves
X=df.loc[:,df.columns!="LABEL"]

m=1 # A chosen exoplanet host star's index for plott
n=100 # A chosen non-exoplanet host star's index

Xtrain =X
Y =Y.to_numpy()
Xtrain=Xtrain.to_numpy()
plt.figure(5)
plt.plot(Xtrain[1])


Xtrain=Xtrain[:,:1+int((len(Xtrain[0])-1)/2)]  #trim the Xtrain data down to about 1600 datapoints instead of 3000+

In [ ]:
print(Xtrain.shape)
print(Y.shape)
Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],1,Xtrain.shape[1]))
print(Xtrain.shape)
print(Y)


In [ ]:
from keras.models import Sequential,Model
from keras.layers import Dense
from keras.layers import LSTM,SimpleRNN
from keras.layers import Dropout,GRU,Flatten,Input
from keras.layers import TimeDistributed,Reshape
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential, load_model
from tqdm import tqdm
from keras.layers.advanced_activations import LeakyReLU
import tensorflow as tf
from keras.optimizers import Adam,Adamax,Nadam
adam = Adam(learning_rate = 3E-2)

In [ ]:

#the generator for the GAN
def create_generator():
    generator=Sequential()
    generator.add(Dense(units=512,input_dim=100))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=1024))
    generator.add(LeakyReLU(0.2))
    
    generator.add(Dense(units=Xtrain.shape[2]))
    
    generator.add(Reshape((1,Xtrain.shape[2])))
    
    generator.compile(loss='binary_crossentropy', optimizer=adam)
    return generator
g = create_generator()
g.summary()

# the discriminator for the GAN
def create_discriminator():
    model = Sequential()
    model.add(LSTM(units =256,input_shape=(1,Xtrain.shape[2]),return_sequences = True))
    model.add(Dropout(0.3))
    model.add(LSTM(units =256,return_sequences = True))
    model.add(Dropout(0.3))
    model.add(LSTM(units =128,return_sequences = False))
    model.add(Dropout(0.3))
    model.add(Dense(units =1,activation = "sigmoid"))
    model.compile(loss='binary_crossentropy',optimizer="rmsprop",metrics = ["accuracy"])
    return model
d =create_discriminator()
d.summary()

def create_gan(discriminator, generator):
    discriminator.trainable=False
    gan_input = Input(shape=(100,))
    x = generator(gan_input)
    
    gan_output= discriminator(x)
    gan= Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='rmsprop')
    return gan

gan = create_gan(d,g)
gan.summary()


def training(epochs=1, batch_size=128):
    
    #Loading the data
    X_train, y_train = Xtrain,Y
    batch_count = X_train.shape[0] / batch_size
    X_train = np.reshape(X_train,(X_train.shape[0],1,X_train.shape[2]))
    # Creating GAN
    generator= create_generator()
    discriminator= create_discriminator()
    gan = create_gan(discriminator, generator)
    
    for e in range(1,epochs+1 ):
        print("Epoch %d" %e)
        for _ in tqdm(range(batch_size)):
        #generate  random noise as an input  to  initialize the  generator
            noise= np.random.normal(0,1, [batch_size, 100])
            
            # Generate fake MNIST images from noised input
            generated_images = generator.predict(noise)
            
            # Get a random set of  real images
            image_batch =X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]
            
            #Construct different batches of  real and fake data 
            X= np.concatenate([image_batch, generated_images])
            X = np.reshape(X,(X.shape[0],1,X_train.shape[2]))
            # Labels for generated and real data
            y_dis=np.zeros(2*batch_size)
            y_dis[:batch_size]=0.9
            
            #Pre train discriminator on  fake and real data  before starting the gan. 
            discriminator.trainable=True
            discriminator.train_on_batch(X, y_dis)
            
            #Tricking the noised input of the Generator as real data
            noise= np.random.normal(0,1, [batch_size, 100])
            y_gen = np.ones(batch_size)
            
            # During the training of gan, 
            # the weights of discriminator should be fixed. 
            #We can enforce that by setting the trainable flag
            discriminator.trainable=False
            
            #training  the GAN by alternating the training of the Discriminator 
            #and training the chained GAN model with Discriminator’s weights freezed.
            gan.train_on_batch(noise, y_gen)
            
        if e == 1 or e % 5 == 0:
            print(generated_images.shape)
        
            for i in range(0,batch_size):
                plt.figure()
                plt.plot(generated_images[i][0])
            plt.show()
        if e==epochs:
            for i in range(0,batch_size,2):
                plt.figure()
                plt.plot(generated_images[i])
            generator.save("generator.h5")
            generator.save_weights("generator.h5")
            gan.save("gan")
            discriminator.save("discriminator.h5")
training(200,10)
    

EXPORT THE MODEL

In [ ]:
model = load_model("gan")
print(model.summary())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
#Here make sure the layer names match the imported GAN Models layers
new_model = Model(model.get_layer("sequential_7").input,model.get_layer("sequential_7").output)
new_model.load_weights("generator_weights2.h5")

print(new_model.summary())



batch_size = 5
noise = np.random.normal(0,1,[batch_size,100])
test = new_model.predict(noise)
def printer():
    for i in range(0,batch_size):
        plt.figure()
        
        plt.plot(test[i][0])
    plt.show()
printer()

# CLASSIFIER MODEL, Run cells below to train the LSTM and make predictions

In [ ]:
df=pd.read_csv('../input/kepler-labelled-time-series-data/exoTrain.csv')
Y=df.loc[:,df.columns=="LABEL"]
X=df.loc[:,df.columns!="LABEL"]
Xtrain =X
Xtrain = Xtrain.to_numpy()
Y =Y.to_numpy()


Data Preprocessing

In [ ]:
from scipy.fftpack import fft,ifft
#standard data preprocessing before feeding it to the LSTM 
def preprocess(Xtrain):
    print('Applying Fourier Transform...')


    plt.figure(5)
    plt.plot(Xtrain[1])
    Xtrain=np.abs(fft(Xtrain,n=len(Xtrain[m]),axis=1))


    print(Xtrain,Xtrain.shape)

    Xtrain=Xtrain[:,:1+int((len(Xtrain[0])-1)/2)]

    plt.figure(1)
    save_train_m = Xtrain[m]
    plt.plot(Xtrain[m],'r')
    plt.title('After FFT (for an exoplanet star)')
    plt.xlabel('Frequency')
    plt.ylabel('Feature value')
    plt.show()

    #### Normalizing

    from sklearn.preprocessing import normalize

    print('Normalizing...')
    Xtrain=normalize(Xtrain)
   
    plt.figure(2)
    plt.plot(Xtrain[m],'r')
    plt.title('After FFT,Normalization (for an exoplanet star)')
    plt.xlabel('Frequency')
    plt.ylabel('Feature value')
    plt.show()




    #### Applying Gaussian Filter

    from scipy import ndimage

    print('Applying Gaussian filter...')
    Xtrain=ndimage.filters.gaussian_filter(Xtrain,sigma=10)
    
    plt.figure(3)
    plt.plot(Xtrain[m],'r')
    plt.title('After FFT,Normalization and Gaussian filtering (for an exoplanet star)')
    plt.xlabel('Frequency')
    plt.ylabel('Feature value')
    plt.show()




    #### Scaling down the data

    from sklearn.preprocessing import MinMaxScaler

    print('Applying MinMaxScaler...')
    scaler=MinMaxScaler(feature_range=(0,1))
    Xtrain=scaler.fit_transform(Xtrain)
    
    plt.figure(4)
    plt.plot(Xtrain[m],'r')
    plt.title('After FFT,Normalization, Gaussian filtering and scaling (for an exoplanet star)')
    plt.xlabel('Frequency')
    plt.ylabel('Feature value')
    plt.show()
    return Xtrain
Xtrain = preprocess(Xtrain)

In [ ]:
print(Xtrain.shape)
print(Y.shape)

Xtrain = np.reshape(Xtrain,(Xtrain.shape[0],1,Xtrain.shape[1]))
print(Xtrain.shape)
print(Y)

for i in range(len(Y)):
    if Y[i]==1:
        Y[i]=0
for i in range(len(Y)):
    if Y[i]==2:
        Y[i]=1
print(Y)

In [ ]:

Classifier = Sequential()
Classifier.add(LSTM(units =256,input_shape=(1,Xtrain.shape[2]),return_sequences = True))
Classifier.add(Dropout(0.3))
Classifier.add(LSTM(units =256,return_sequences = True))
Classifier.add(Dropout(0.3))
Classifier.add(LSTM(units =128,return_sequences = False))
Classifier.add(Dropout(0.3))
Classifier.add(Dense(units =1,activation = "sigmoid"))
Classifier.compile(loss='binary_crossentropy',optimizer="adam",metrics = ["accuracy"])

Classifier.fit(Xtrain,Y,epochs = 10)

Classifier.save("classifier.h5")

In [ ]:
#quick sanity check after the model is trained
print(Xtrain[23])
plt.plot(Xtrain[34][0])
plt.show()
print(Y[23])

print(Xtrain[0].shape)
y = Classifier.predict(Xtrain)
print("prediction: ",y[322])
print("actual label: ",Y[322])

In [ ]:
def preprocess_GAN(Xtrain):
    print('Applying Fourier Transform...')
    Xtrain=np.abs(fft(Xtrain,n=len(Xtrain[m]),axis=1))
    print(Xtrain,Xtrain.shape)

    #### Normalizing
    from sklearn.preprocessing import normalize
    print('Normalizing...')
    Xtrain=normalize(Xtrain)
   
    #### Applying Gaussian Filter
    from scipy import ndimage
    print('Applying Gaussian filter...')
    Xtrain=ndimage.filters.gaussian_filter(Xtrain,sigma=10)
    
    
    #### Scaling down the data
    from sklearn.preprocessing import MinMaxScaler
    print('Applying MinMaxScaler...')
    scaler=MinMaxScaler(feature_range=(0,1))
    Xtrain=scaler.fit_transform(Xtrain)
    
    plt.figure(4)
    plt.plot(Xtrain[m],'r')
    plt.show()
    return Xtrain

In [ ]:
print(test.shape)
batch_size = 5
noise = np.random.normal(0,1,[batch_size,100])
test = new_model.predict(noise)
X_val = np.reshape(test,(test.shape[0],test.shape[2]))
X_val = preprocess_GAN(X_val)
X_val = np.reshape(X_val,(X_val.shape[0],1,X_val.shape[1]))
plt.plot(X_val[1][0])
plt.show()
y_val = Classifier.predict(X_val)
print(y_val)